In [ ]:
!pip install pandas
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
## Isi nama dan NIM seluruh anggota kelompok di sini
#   Nama    NIM
#   Razita Amalina              1301200283
#   Gayuh Putri Andyani         1301204050
#   Yunolva Anis                1301204096
#   Aufa Mutia                  1301204233
#   
#   

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree


# Load Dataset
Inisialisasi variable, masukan NIM di sini untuk random state.

Gunakan NIM salah satu anggota kelompok.



In [ ]:
filepath = "/content/seeds_dataset.csv"

# Isi nim anda di sini
nim = 1301204096

Load dataset, cek konten data

Target kelas: **Category** 

In [ ]:
df = pd.read_csv(filepath, sep="\t")
df.sample(10, random_state=nim)

df.describe()

,Area,Perimeter,Compactness,Kernel_len,Kernel_width,Assymetry,Groove_len,Category
count,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000
mean,14.847524,14.559286,0.870999,5.628533,3.258605,3.700201,5.408071,2.000000
std,2.909699,1.305959,0.023629,0.443063,0.377714,1.503557,0.491480,0.818448
min,10.590000,12.410000,0.808100,4.899000,2.630000,0.765100,4.519000,1.000000
25%,12.270000,13.450000,0.856900,5.262250,2.944000,2.561500,5.045000,1.000000
50%,14.355000,14.320000,0.873450,5.523500,3.237000,3.599000,5.223000,2.000000
75%,17.305000,15.715000,0.887775,5.979750,3.561750,4.768750,5.877000,3.000000
max,21.180000,17.250000,0.918300,6.675000,4.033000,8.456000,6.550000,3.000000


Training/test split **60/40**

In [ ]:
# X: semua kolom, Y: kolom terakhir (category)
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
# print(x)

# print(y)


x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.6, random_state=nim)
# We perform bootstrap on the train data
# Keep test data for final accuracy testing
len(x_train), len(x_test),len(y_train)


      Area  Perimeter  Compactness  Kernel_len  Kernel_width  Assymetry  \
171  11.55      13.10       0.8455       5.167         2.845      6.715   
160  12.54      13.67       0.8425       5.451         2.879      3.082   
59   12.11      13.47       0.8392       5.159         3.032      1.502   
186  11.81      13.45       0.8198       5.413         2.716      4.898   
98   18.17      16.26       0.8637       6.271         3.512      2.853   
..     ...        ...          ...         ...           ...        ...   
135  15.38      14.66       0.8990       5.477         3.465      3.600   
168  11.35      13.12       0.8291       5.176         2.668      4.337   
63   13.22      13.84       0.8680       5.395         3.070      4.157   
90   20.10      16.99       0.8746       6.581         3.785      1.955   
172  11.27      12.97       0.8419       5.088         2.763      4.309   

     Groove_len  
171       4.956  
160       5.491  
59        4.519  
186       5.352  
98       

# Random Forest
Build a Random Forest, consisting of 20 Decision Tree classifer, with the following steps:

1. Prepare Bootstrap data (choose randomly from the *training data*, use sampling with replacement)
2. Choose 2-4 random features/atributes for each tree
3. Build Decision Tree classifier using step (1) and (2)
4. Calculate OOB score within the tree only

**NOTE**
This whole exercise has been simplified.
- Tree can be created using a library, therefore you may not be able control the feature selection on node level
- Random features are only selected in the beginning of each tree.
- OOB score is only calculated within its own tree instead.
- Typically Random Forest don't use train/test split and can be validated using OOB error. We simplify this to make it easier to test.
- You are not allowed to use a Random Forest or Bagging library.


In [ ]:
np.random.seed(nim)
n_tree = 20

forest = []
bootstrap_columns = []
for i in range(n_tree):
    # 1. Bootstrapping
    # select random rows

    #index rows akan dipilih secara random berdasarkan x_train
    rows =np.random.choice(x_train.shape[0], size=x_train.shape[0], replace=True)
    x_bts = x_train.iloc[rows, :]
    y_bts = y_train.iloc[rows]
    # x_bootstrap = x_train.iloc[rows]
    # print(x_bootstrap)
    # y_bootstrap = y_train.iloc[rows]

    # Out of Bag data (data yang tidak dipilih)
    # Merupakan sebuah metode yang digunakan untuk mengevaluasi metode Bagging dengan menggunakan data yang tidak masuk ke dalam bootstraping
    non_selected_rows = np.setdiff1d(np.arange(x_train.shape[0]), rows)
    x_oob =x_train.iloc[non_selected_rows, :]
    y_oob = y_train.iloc[non_selected_rows]
    n_oob = len(non_selected_rows)

    # 2. Select random features (make sure to keep track the selected columns)
    # Check for any repeated combinations
    feature_taken = np.random.randint(2, 4)
    cols = np.random.choice(x_train.shape[1], size=feature_taken, replace=False)
    bootstrap_columns.append(cols)
    print(x_train.columns[cols])

    # filter based on the selected columns
    x_bts = x_bts.iloc[:,cols]

    # 3. Build decision tree
    # Let it grow, no pruning
    from sklearn.tree import DecisionTreeClassifier
    tree_model = DecisionTreeClassifier(max_depth = None)
    tree_model.fit(x_bts, y_bts)

    # Add it to the forest
    forest.append(tree_model)

    # 4. Calculate oob score only within its own tree
    # ....
    from sklearn.metrics import accuracy_score
    oob_pred = tree_model.predict(x_oob.iloc[:, cols])
    oob_score = accuracy_score(y_oob, oob_pred)
    print("Number of Root ", n_oob, "OOB Score : ", oob_score)



Index(['Perimeter', 'Compactness'], dtype='object')
Number of Root  45 OOB Score :  0.8666666666666667
Index(['Groove_len', 'Area', 'Assymetry'], dtype='object')
Number of Root  47 OOB Score :  0.8936170212765957
Index(['Assymetry', 'Groove_len'], dtype='object')
Number of Root  46 OOB Score :  0.8913043478260869
Index(['Assymetry', 'Perimeter', 'Kernel_len'], dtype='object')
Number of Root  43 OOB Score :  0.9069767441860465
Index(['Groove_len', 'Kernel_len'], dtype='object')
Number of Root  50 OOB Score :  0.96
Index(['Groove_len', 'Perimeter'], dtype='object')
Number of Root  41 OOB Score :  0.9024390243902439
Index(['Groove_len', 'Kernel_width'], dtype='object')
Number of Root  52 OOB Score :  0.8846153846153846
Index(['Area', 'Kernel_len', 'Kernel_width'], dtype='object')
Number of Root  46 OOB Score :  0.717391304347826
Index(['Kernel_width', 'Compactness'], dtype='object')
Number of Root  45 OOB Score :  0.7777777777777778
Index(['Assymetry', 'Area', 'Kernel_len'], dtype='object

In [ ]:
print(x_oob)

      Area  Perimeter  Compactness  Kernel_len  Kernel_width  Assymetry  \
171  11.55      13.10       0.8455       5.167         2.845     6.7150   
186  11.81      13.45       0.8198       5.413         2.716     4.8980   
0    15.26      14.84       0.8710       5.763         3.312     2.2210   
66   14.34      14.37       0.8726       5.630         3.190     1.3130   
123  18.43      15.97       0.9077       5.980         3.771     2.9840   
179  12.21      13.47       0.8453       5.357         2.893     1.6610   
150  11.83      13.23       0.8496       5.263         2.840     5.1950   
74   16.82      15.51       0.8786       6.017         3.486     4.0040   
32   14.09      14.41       0.8529       5.717         3.186     3.9200   
22   15.88      14.90       0.8988       5.618         3.507     0.7651   
145  11.21      13.13       0.8167       5.279         2.687     6.1690   
108  19.94      16.92       0.8752       6.675         3.763     3.2520   
36   16.20      15.27    

In [ ]:
print(y_oob)

171    3
186    3
0      1
66     1
123    2
179    3
150    3
74     2
32     1
22     1
145    3
108    2
36     1
153    3
202    3
95     2
115    2
82     2
206    3
42     1
131    2
119    2
44     1
180    3
130    2
164    3
37     1
198    3
111    2
144    3
88     2
181    3
68     1
77     2
67     1
47     1
196    3
55     1
15     1
54     1
26     1
97     2
100    2
102    2
96     2
136    2
124    2
109    2
33     1
168    3
90     2
Name: Category, dtype: int64


# Prediction and Voting
Run prediction on x_test data for each tree

Also calculate accuracy per tree here

In [ ]:
all_preds = []
for i, tree_model in enumerate(forest):
    # Each tree has different column requirements
    # Make sure you use the correct columns for each tree
    x_test_filtered = x_test.iloc[:,bootstrap_columns[i]]

    predictions = tree_model.predict(x_test_filtered)

    # Might as well calculate accuracy per tree here...

    acc = accuracy_score(y_test, predictions)
    print(f"Tree {i+1} Test Accuracy: {acc:.4f}")  

    # add it to all_preds for voting later
    all_preds.append(predictions)

all_preds = np.array(all_preds)

Tree 1 Test Accuracy: 0.8214
Tree 2 Test Accuracy: 0.9048
Tree 3 Test Accuracy: 0.7738
Tree 4 Test Accuracy: 0.8214
Tree 5 Test Accuracy: 0.8571
Tree 6 Test Accuracy: 0.9167
Tree 7 Test Accuracy: 0.9286
Tree 8 Test Accuracy: 0.8810
Tree 9 Test Accuracy: 0.8571
Tree 10 Test Accuracy: 0.8333
Tree 11 Test Accuracy: 0.9405
Tree 12 Test Accuracy: 0.8333
Tree 13 Test Accuracy: 0.9167
Tree 14 Test Accuracy: 0.8810
Tree 15 Test Accuracy: 0.7857
Tree 16 Test Accuracy: 0.8214
Tree 17 Test Accuracy: 0.8333
Tree 18 Test Accuracy: 0.8571
Tree 19 Test Accuracy: 0.8452
Tree 20 Test Accuracy: 0.8333


### Perform voting on the predictions from the Random Forest

In [ ]:
# Insert voting code here
# ...
# ...


voted_predictions = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=all_preds)
# Calculate accuracy of Test data on the Forest
acc = accuracy_score(y_test, voted_predictions)
acc



0.9523809523809523

After you're done with all this, please check the excel file and put the answers for every column there.